In [ ]:
# deflactor trimestral fred https://fred.stlouisfed.org/series/CPALTT01MXQ657N 
#

import pandas as pd
from pathlib import Path


In [ ]:
# Vector trimestral del deflactor nacional (base 2005)
deflactor_mexico = [
    100.00, 100.52, 101.25, 102.34, 103.11, 103.22, 104.13, 105.47,
    106.51, 106.68, 107.82, 109.43, 110.60, 111.64, 113.14, 115.02,
    116.38, 116.44, 117.15, 118.06, 119.92, 120.15, 120.91, 122.37,
    123.36, 123.68, 124.58, 126.11, 127.28, 127.89, 129.23, 131.02,
    132.22, 132.30, 133.22, 134.41, 135.61, 135.83, 136.91, 138.25,
    138.86, 138.76, 139.51, 140.24, 141.01, 141.13, 142.17, 143.94,
    146.12, 147.27, 148.87, 151.10, 153.07, 153.03, 154.51, 156.41,
    157.65, 158.12, 158.83, 159.87, 161.42, 161.27, 162.59, 164.84,
    166.92, 168.12, 170.36, 173.84, 176.63, 179.02, 182.26, 185.73,
    188.76, 188.80, 190.23, 192.51, 195.96, 196.86, 199.11, 201.76,
    203.68, 205.14
]

# INPC Mensual México (Base 2018=100)
# Fuente: INEGI (Serie histórica unificada)
# Periodo: Enero 2005 a Diciembre 2023

inpc_historico = [
    # 2005 (Ene-Dic)
    58.625, 58.790, 59.049, 59.259, 59.108, 59.045, 59.277, 59.349, 59.585, 59.728, 60.158, 60.468,
    # 2006
    60.824, 60.916, 61.000, 61.096, 60.817, 60.871, 61.037, 61.348, 61.966, 62.238, 62.562, 62.926,
    # 2007
    63.293, 63.473, 63.613, 63.646, 63.336, 63.411, 63.679, 64.027, 64.530, 64.786, 65.247, 65.522,
    # 2008
    65.823, 66.012, 66.488, 66.642, 66.568, 66.840, 67.216, 67.608, 68.066, 68.528, 69.243, 69.619,
    # 2009
    69.782, 69.937, 70.360, 70.613, 70.407, 70.611, 70.767, 70.939, 71.282, 71.558, 71.790, 72.087,
    # 2010
    72.871, 73.294, 73.818, 73.582, 73.118, 73.048, 73.210, 73.417, 73.791, 74.204, 74.838, 75.210,
    # 2011
    75.337, 75.623, 75.768, 75.797, 75.234, 75.236, 75.600, 75.723, 75.913, 76.375, 77.202, 78.220,
    # 2012
    78.683, 78.845, 78.892, 78.625, 78.375, 78.736, 79.176, 79.414, 79.764, 80.173, 80.735, 81.077,
    # 2013
    81.401, 81.799, 82.399, 82.545, 82.270, 82.219, 82.193, 82.422, 82.736, 83.136, 83.911, 84.392,
    # 2014
    85.143, 85.360, 85.592, 85.429, 85.155, 85.303, 85.538, 85.802, 86.183, 86.657, 87.464, 87.893,
    # 2015
    87.818, 87.986, 88.347, 88.118, 87.676, 87.828, 87.959, 88.148, 88.479, 88.932, 89.424, 89.794,
    # 2016
    90.134, 90.531, 90.669, 90.379, 90.061, 90.163, 90.400, 90.655, 91.205, 91.761, 92.479, 92.905,
    # 2017
    94.490, 95.035, 95.617, 95.733, 95.616, 95.857, 96.220, 96.691, 97.000, 97.614, 98.618, 99.199,
    # 2018 (Año Base: Julio 2da Quincena = 100, Promedio Julio ~99.9)
    99.726, 100.106, 100.430, 100.088, 99.928, 100.060, 100.601, 101.187, 101.611, 102.138, 103.008, 103.725,
    # 2019
    103.818, 103.784, 104.185, 104.239, 104.020, 104.085, 104.479, 104.457, 104.733, 105.302, 106.155, 106.745,
    # 2020
    107.256, 107.708, 107.650, 106.562, 106.969, 107.555, 107.943, 108.365, 108.614, 109.273, 109.366, 109.779,
    # 2021
    110.722, 111.423, 112.345, 112.697, 112.923, 113.525, 114.195, 114.414, 115.123, 116.088, 117.415, 117.840,
    # 2022
    118.532, 119.514, 120.693, 121.348, 121.564, 122.585, 123.492, 124.356, 125.130, 125.845, 126.578, 127.061,
    # 2023
    127.930, 128.667, 129.018, 128.990, 128.704, 128.837, 129.458, 130.172, 130.749, 131.248, 132.088, 132.964
]

inpc_historico = [
    # --- 2005 ---
    58.625, 58.790, 59.049, 59.259, 59.108, 59.045, 59.277, 59.349, 59.585, 59.728, 60.158, 60.468,
    # --- 2006 ---
    60.824, 60.916, 61.000, 61.096, 60.817, 60.871, 61.037, 61.348, 61.966, 62.238, 62.562, 62.926,
    # --- 2007 ---
    63.293, 63.473, 63.613, 63.646, 63.336, 63.411, 63.679, 64.027, 64.530, 64.786, 65.247, 65.522,
    # --- 2008 ---
    65.823, 66.012, 66.488, 66.642, 66.568, 66.840, 67.216, 67.608, 68.066, 68.528, 69.243, 69.619,
    # --- 2009 ---
    69.782, 69.937, 70.360, 70.613, 70.407, 70.611, 70.767, 70.939, 71.282, 71.558, 71.790, 72.087,
    # --- 2010 ---
    72.871, 73.294, 73.818, 73.582, 73.118, 73.048, 73.210, 73.417, 73.791, 74.204, 74.838, 75.210,
    # --- 2011 ---
    75.337, 75.623, 75.768, 75.797, 75.234, 75.236, 75.600, 75.723, 75.913, 76.375, 77.202, 78.220,
    # --- 2012 ---
    78.683, 78.845, 78.892, 78.625, 78.375, 78.736, 79.176, 79.414, 79.764, 80.173, 80.735, 81.077,
    # --- 2013 ---
    81.401, 81.799, 82.399, 82.545, 82.270, 82.219, 82.193, 82.422, 82.736, 83.136, 83.911, 84.392,
    # --- 2014 ---
    85.143, 85.360, 85.592, 85.429, 85.155, 85.303, 85.538, 85.802, 86.183, 86.657, 87.464, 87.893,
    # --- 2015 ---
    87.818, 87.986, 88.347, 88.118, 87.676, 87.828, 87.959, 88.148, 88.479, 88.932, 89.424, 89.794,
    # --- 2016 ---
    90.134, 90.531, 90.669, 90.379, 90.061, 90.163, 90.400, 90.655, 91.205, 91.761, 92.479, 92.905,
    # --- 2017 ---
    94.490, 95.035, 95.617, 95.733, 95.616, 95.857, 96.220, 96.691, 97.000, 97.614, 98.618, 99.199,
    # --- 2018 ---
    99.726, 100.106, 100.430, 100.088, 99.928, 100.060, 100.601, 101.187, 101.611, 102.138, 103.008, 103.725,
    # --- 2019 ---
    103.818, 103.784, 104.185, 104.239, 104.020, 104.085, 104.479, 104.457, 104.733, 105.302, 106.155, 106.745,
    # --- 2020 ---
    107.256, 107.708, 107.650, 106.562, 106.969, 107.555, 107.943, 108.365, 108.614, 109.273, 109.366, 109.779,
    # --- 2021 ---
    110.722, 111.423, 112.345, 112.697, 112.923, 113.525, 114.195, 114.414, 115.123, 116.088, 117.415, 117.840,
    # --- 2022 ---
    118.532, 119.514, 120.693, 121.348, 121.564, 122.585, 123.492, 124.356, 125.130, 125.845, 126.578, 127.061,
    # --- 2023 ---
    127.930, 128.667, 129.018, 128.990, 128.704, 128.837, 129.458, 130.172, 130.749, 131.248, 132.088, 132.964,
    # --- 2024 (Actualizado) ---
    133.555, 133.681, 134.065, 134.336, 134.087, 134.594, 136.003, 136.013, 136.080, 136.828, 137.424, 137.949,
    # --- 2025 (Actualizado hasta Octubre) ---
    138.343, 138.726, 139.161, 139.620, 140.012, 140.405, 140.780, 140.867, 141.197, 141.708
]

inpc_historico_trimestral = [
    # 2005
    58.821, 59.137, 59.404, 60.118,
    # 2006
    60.913, 60.928, 61.517, 62.575,
    # 2007
    63.460, 63.464, 63.745, 65.185,
    # 2008
    66.108, 66.683, 67.630, 69.130,
    # 2009
    70.026, 70.544, 70.996, 71.812,
    # 2010
    73.328, 73.249, 73.805, 74.751,
    # 2011
    75.576, 75.422, 75.745, 77.266,
    # 2012
    78.807, 78.579, 79.451, 80.662,
    # 2013
    81.866, 82.345, 82.450, 83.813,
    # 2014
    85.365, 85.296, 85.841, 87.338,
    # 2015
    88.050, 87.874, 88.195, 89.383,
    # 2016
    90.445, 90.201, 90.753, 92.382,
    # 2017
    95.047, 95.735, 96.637, 98.477,
    # 2018
    100.087, 100.025, 101.133, 102.957,
    # 2019
    103.929, 104.115, 104.556, 106.067,
    # 2020
    107.538, 107.029, 108.307, 109.473,
    # 2021
    111.497, 113.048, 114.577, 117.114,
    # 2022
    119.580, 121.832, 124.326, 126.495,
    # 2023
    128.538, 128.844, 130.126, 132.100,
    # 2024
    133.767, 134.339, 136.032, 137.400,
    # 2025
    138.743, 140.012, 140.948
]

def construir_deflactor_dataframe(deflactor, periodo_inicio="2005Q1"):
    """Construye un DataFrame con year/quarter y el deflactor para el merge."""
    fechas = pd.period_range(start=periodo_inicio, periods=len(deflactor), freq="Q")
    df_def = pd.DataFrame({"deflactor": deflactor, "fecha_idx": fechas})
    df_def["year"] = df_def["fecha_idx"].dt.year
    df_def["quarter"] = df_def["fecha_idx"].dt.quarter
    return df_def[["year", "quarter", "deflactor"]]


In [ ]:
def deflactar_nacional(ruta_archivo, columnas_monetarias, deflactor, periodo_inicio="2005Q1"):
    """
    Lee un archivo de Datos Nacionales, agrega el deflactor y genera columnas def_ para cada variable monetaria.
    Guarda el resultado con el sufijo _deflactado y regresa el DataFrame y la ruta de salida.
    """
    df = pd.read_csv(ruta_archivo)
    return deflactar_base(df, columnas_monetarias, deflactor, ruta_archivo, periodo_inicio)


def deflactar_base(df, columnas_monetarias, deflactor, ruta_archivo=None, periodo_inicio="2005Q1"):
    """
    Recibe un DataFrame con year/quarter, lista de columnas monetarias y vector de deflactor.
    Agrega columnas def_ y, si se provee ruta_archivo, guarda CSV con sufijo _deflactado.
    """
    df = df.copy()
    df["year"] = df["year"].astype(int)
    df["quarter"] = df["quarter"].astype(int)

    df_deflactor = construir_deflactor_dataframe(deflactor, periodo_inicio)
    df_final = pd.merge(df, df_deflactor, on=["year", "quarter"], how="left")

    if df_final["deflactor"].isna().any():
        raise ValueError("Faltan valores del deflactor para algunos trimestres.")

    faltantes = [col for col in columnas_monetarias if col not in df_final.columns]
    if faltantes:
        raise ValueError(f"Las columnas no est?n en el archivo: {faltantes}")

    for col in columnas_monetarias:
        df_final[f"def_{col}"] = (df_final[col] / df_final["deflactor"]) * 100

    archivo_salida = None
    if ruta_archivo:
        nombre_archivo = Path(ruta_archivo)
        archivo_salida = nombre_archivo.with_name(f"{nombre_archivo.stem}_deflactado.csv")
        df_final.to_csv(archivo_salida, index=False)

    return df_final, archivo_salida


In [ ]:
# Ejemplo de uso con variables monetarias
archivo_nacional = "20251205_Nacional.csv"
variables_a_deflactar = [
    "masa_salarial_total",
    "masa_salarial_hombres",
    "masa_salarial_mujeres",
    "ing_hora_hombres",
    "ing_hora_mujeres",
    "ing_mensual_hombres",
    "ing_mensual_mujeres",
    "ing_prim_inc_hombres",
    "ing_prim_inc_mujeres",
    "ing_prim_comp_hombres",
    "ing_prim_comp_mujeres",
    "ing_secundaria_hombres",
    "ing_secundaria_mujeres",
    "ing_sup_y_mas_hombres",
    "ing_sup_y_mas_mujeres",
    "ing_hora_primaria_hombres",
    "ing_hora_primaria_mujeres",
    "ing_hora_secundaria_hombres",
    "ing_hora_secundaria_mujeres",
    "ing_hora_superior_hombres",
    "ing_hora_superior_mujeres",
]


df_deflactado, archivo_salida = deflactar_nacional(
    archivo_nacional,
    variables_a_deflactar,
    inpc_historico_trimestral,
)

print(f"Archivo guardado en: {archivo_salida}")
df_deflactado.head()


In [ ]:
def deflactar_estados(ruta_archivo, columnas_monetarias, deflactor, periodo_inicio="2005Q1"):
    """Lee un archivo de Datos por estado, agrega el deflactor y genera columnas def_."""
    df = pd.read_csv(ruta_archivo)
    return deflactar_base(df, columnas_monetarias, deflactor, ruta_archivo, periodo_inicio)

In [ ]:
# Deflactar variables estatales usando el mismo deflactor nacional
archivo_estados = "Resultados/20251205_Estados.csv"
variables_estatales_a_deflactar = [
    "masa_salarial_total",
    "masa_salarial_hombres",
    "masa_salarial_mujeres",
    "ing_hora_hombres",
    "ing_hora_mujeres",
    "ing_mensual_hombres",
    "ing_mensual_mujeres",
    "ing_prim_inc_hombres",
    "ing_prim_inc_mujeres",
    "ing_prim_comp_hombres",
    "ing_prim_comp_mujeres",
    "ing_secundaria_hombres",
    "ing_secundaria_mujeres",
    "ing_sup_y_mas_hombres",
    "ing_sup_y_mas_mujeres",
    "ing_hora_primaria_hombres",
    "ing_hora_primaria_mujeres",
    "ing_hora_secundaria_hombres",
    "ing_hora_secundaria_mujeres",
    "ing_hora_superior_hombres",
    "ing_hora_superior_mujeres",
    "ing_prom_mensual",
]


df_estados_deflactado, archivo_salida_estados = deflactar_estados(
    archivo_estados,
    variables_estatales_a_deflactar,
    inpc_historico_trimestral,
)

print(f"Archivo de estados guardado en: {archivo_salida_estados}")
df_estados_deflactado.head()